# Cleaning raw JSON tweets data scraped using snscrape library

## Introduction

In this notebook, I will be discussing how to clean and pre-process raw JSON data about tweets scraped using the Python library snscrape. The JSON data is finally converted to CSV files to make it easier for analysis.

As an example, I have scraped tweets that contain the hashtag "#FarmersProtest" using snscrape which gives a JSON file about the relevant tweets.

snscrape is a Python library that allows you to scrape tweets easily through the Twitter API without any request limits. I will not be focussing on how to scrape tweets and get the raw JSON tweets data. For an easy-to-follow tutorial on how to use snscrape to scrape tweets through the Twitter API, check out [this Medium blog by Martin Beck](https://betterprogramming.pub/how-to-scrape-tweets-with-snscrape-90124ed006af).

## Importing required libraries

Let's start by importing the required libraries. We will be needing Pandas to load and work with JSON data as well as the json_normalize() function in the pandas.io.json package to perform some transformation functions on JSON data.

In [1]:
# Importing required libraries

import pandas as pd
from collections import Counter
import re
from pandas.io.json import json_normalize
import warnings
warnings.filterwarnings("ignore")

## Read raw JSON tweets data

Next, we load the raw JSON tweets data using the function read_json() available in pandas library. Since we are interested in performing analysis using techniques such as NLP, I have only retained tweets that are in the English language. Next, let's take a look at the first 5 records for the raw JSON data.

In [2]:
# Read JSON file containing tweets data and removce tweets not in English

raw_tweets = pd.read_json(r'dataset/farmers-protest-tweets-2021-03-5.json', lines=True)
raw_tweets = raw_tweets[raw_tweets['lang']=='en']
print("Shape: ", raw_tweets.shape)
raw_tweets.head(5)

Shape:  (417511, 21)


,url,date,content,renderedContent,id,user,outlinks,tcooutlinks,replyCount,retweetCount,...,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,media,retweetedTweet,quotedTweet,mentionedUsers
0,https://twitter.com/ShashiRajbhar6/status/1376...,2021-03-30 03:33:46+00:00,Support 👇\n\n#FarmersProtest,Support 👇\n\n#FarmersProtest,1376739399593910273,"{'username': 'ShashiRajbhar6', 'displayname': ...",[],[],0,0,...,0,1376739399593910273,en,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,None,NaN,None,None
1,https://twitter.com/kaursuk06272818/status/137...,2021-03-30 03:33:23+00:00,Supporting farmers means supporting our countr...,Supporting farmers means supporting our countr...,1376739306287427584,"{'username': 'kaursuk06272818', 'displayname':...",[],[],0,0,...,0,1376739306287427584,en,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,[{'previewUrl': 'https://pbs.twimg.com/media/E...,NaN,None,None
2,https://twitter.com/kaursuk06272818/status/137...,2021-03-30 03:31:00+00:00,Support farmers if you are related to food #St...,Support farmers if you are related to food #St...,1376738704128020488,"{'username': 'kaursuk06272818', 'displayname':...",[],[],0,0,...,0,1376738704128020488,en,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,[{'previewUrl': 'https://pbs.twimg.com/media/E...,NaN,None,None
3,https://twitter.com/SukhdevSingh_/status/13767...,2021-03-30 03:30:45+00:00,#StopHateAgainstFarmers support #FarmersProtes...,#StopHateAgainstFarmers support #FarmersProtes...,1376738640542400518,"{'username': 'SukhdevSingh_', 'displayname': '...",[],[],0,1,...,0,1376738640542400518,en,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,None,NaN,None,None
4,https://twitter.com/Davidmu66668113/status/137...,2021-03-30 03:30:30+00:00,"You hate farmers I hate you, \nif you love the...","You hate farmers I hate you, \nif you love the...",1376738579171344386,"{'username': 'Davidmu66668113', 'displayname':...",[],[],0,0,...,0,1376738579171344386,en,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,None,NaN,None,None


## Normalize 'user' field in raw_tweets

We see that 'raw_tweets' has a nested JSON field named 'user'. This field can be normalized for better analysis using the json_normalize() function in the pandas.io.json library. Essentially, semi-structured JSON data is "normalized" into a flat table.

For more info on how to use json_normalize(), check out [the documentation page for pandas.io.json.json_normalize()](https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.io.json.json_normalize.html).

I have also renamed the fields 'id' to 'userId' and 'url' to 'profileUrl' for it to make more sense and avoid confusion. The fields 'description' and 'linkTcourl' are not important and hence, have been dropped.

Let's take a look at the first 5 records.

In [3]:
# Normalize 'user' field

users = json_normalize(raw_tweets['user'])
users.drop(['description', 'linkTcourl'], axis=1, inplace=True)
users.rename(columns={'id':'userId', 'url':'profileUrl'}, inplace=True)
users.head(5)

,username,displayname,userId,rawDescription,descriptionUrls,verified,created,followersCount,friendsCount,statusesCount,favouritesCount,listedCount,mediaCount,location,protected,linkUrl,profileImageUrl,profileBannerUrl,profileUrl
0,ShashiRajbhar6,Shashi Rajbhar,1015969769760096256,Satya presan 🤔ho Sakta but prajit💪 nhi\njhuth ...,[],False,2018-07-08T14:44:03+00:00,1788,1576,14396,26071,1,254,"Azm Uttar Pradesh, India",False,None,https://pbs.twimg.com/profile_images/135433129...,https://pbs.twimg.com/profile_banners/10159697...,https://twitter.com/ShashiRajbhar6
1,kaursuk06272818,KAUR SUKH🌾ਕੌਰ ਸੁਖ,1332937272581263362,ਜਿਓਣਾ ਕੀ ਸਰੀਰਾਂ ਦਾ ਜੇਕਰ ਹੋਣ ਜ਼ਮੀਰਾਂ ਮਰੀਆਂ 🌼,[],False,2020-11-29T06:40:06+00:00,51,68,1338,3676,0,607,,False,None,https://pbs.twimg.com/profile_images/133295149...,https://pbs.twimg.com/profile_banners/13329372...,https://twitter.com/kaursuk06272818
2,kaursuk06272818,KAUR SUKH🌾ਕੌਰ ਸੁਖ,1332937272581263362,ਜਿਓਣਾ ਕੀ ਸਰੀਰਾਂ ਦਾ ਜੇਕਰ ਹੋਣ ਜ਼ਮੀਰਾਂ ਮਰੀਆਂ 🌼,[],False,2020-11-29T06:40:06+00:00,51,68,1338,3676,0,607,,False,None,https://pbs.twimg.com/profile_images/133295149...,https://pbs.twimg.com/profile_banners/13329372...,https://twitter.com/kaursuk06272818
3,SukhdevSingh_,Sukhdev Singh,1308356658582618112,Just a part of my society . Social and Politic...,[],False,2020-09-22T10:45:27+00:00,2595,3314,3281,3533,0,519,"Punjab, India",False,None,https://pbs.twimg.com/profile_images/130835702...,https://pbs.twimg.com/profile_banners/13083566...,https://twitter.com/SukhdevSingh_
4,Davidmu66668113,tera jija 🤨🚩🇺🇸,1357311756532649985,dream boy 🌪🌍🔥💯,[],False,2021-02-04T12:55:36+00:00,18,286,347,520,0,3,,False,None,https://pbs.twimg.com/profile_images/137600703...,https://pbs.twimg.com/profile_banners/13573117...,https://twitter.com/Davidmu66668113


## Create users DF

Next, let's create the final DataFrame for Twitter users who tweeted using the hashtag "#FarmersProtest". I have also dropped duplicate records from the DataFrame based on the field 'userID' as each user must have a unique user ID.

Let's take a look at the shape and first 5 records for the final DataFrame for the Twitter users.

In [4]:
# Create DataFrame and remove duplicates

users = pd.DataFrame(users)
users.drop_duplicates(subset=['userId'], inplace=True)
print("Shape: ", users.shape)
users.head(5)

Shape:  (93223, 19)


,username,displayname,userId,rawDescription,descriptionUrls,verified,created,followersCount,friendsCount,statusesCount,favouritesCount,listedCount,mediaCount,location,protected,linkUrl,profileImageUrl,profileBannerUrl,profileUrl
0,ShashiRajbhar6,Shashi Rajbhar,1015969769760096256,Satya presan 🤔ho Sakta but prajit💪 nhi\njhuth ...,[],False,2018-07-08T14:44:03+00:00,1788,1576,14396,26071,1,254,"Azm Uttar Pradesh, India",False,None,https://pbs.twimg.com/profile_images/135433129...,https://pbs.twimg.com/profile_banners/10159697...,https://twitter.com/ShashiRajbhar6
1,kaursuk06272818,KAUR SUKH🌾ਕੌਰ ਸੁਖ,1332937272581263362,ਜਿਓਣਾ ਕੀ ਸਰੀਰਾਂ ਦਾ ਜੇਕਰ ਹੋਣ ਜ਼ਮੀਰਾਂ ਮਰੀਆਂ 🌼,[],False,2020-11-29T06:40:06+00:00,51,68,1338,3676,0,607,,False,None,https://pbs.twimg.com/profile_images/133295149...,https://pbs.twimg.com/profile_banners/13329372...,https://twitter.com/kaursuk06272818
3,SukhdevSingh_,Sukhdev Singh,1308356658582618112,Just a part of my society . Social and Politic...,[],False,2020-09-22T10:45:27+00:00,2595,3314,3281,3533,0,519,"Punjab, India",False,None,https://pbs.twimg.com/profile_images/130835702...,https://pbs.twimg.com/profile_banners/13083566...,https://twitter.com/SukhdevSingh_
4,Davidmu66668113,tera jija 🤨🚩🇺🇸,1357311756532649985,dream boy 🌪🌍🔥💯,[],False,2021-02-04T12:55:36+00:00,18,286,347,520,0,3,,False,None,https://pbs.twimg.com/profile_images/137600703...,https://pbs.twimg.com/profile_banners/13573117...,https://twitter.com/Davidmu66668113
5,Abhimanyu_1987,Abhimanyu 🌏 🇮🇳,2918610912,Seeker...,[],False,2014-12-04T13:29:54+00:00,173,41,8954,16364,19,112,"Jaipur,Rajasthan,India",False,None,https://pbs.twimg.com/profile_images/125684524...,https://pbs.twimg.com/profile_banners/29186109...,https://twitter.com/Abhimanyu_1987


## Create tweets DF

Next, we will transform the 'raw_tweets' DataFrame to obtain a DataFrame for tweets that contain the hashtag "#FarmersProtest". A new field, 'userId' is added which corresponds to the unique ID of the user who posted the particular tweet.

Next, I have retained only the important fields and renamed the fields 'id' to 'tweetId' and 'url' to 'tweetUrl' for it to make more sense and avoid confusion.

Let's take a look at the first 5 records of this DataFrame.

In [5]:
# Transform 'raw_tweets' DataFrame

# Add column for 'userId'
user_id = []
for user in raw_tweets['user']:
    uid = user['id']
    user_id.append(uid)
raw_tweets['userId'] = user_id

# Remove less important columns
cols = ['url', 'date', 'renderedContent', 'id', 'userId', 'replyCount', 'retweetCount', 'likeCount', 'quoteCount', 'source', 'media', 'retweetedTweet', 'quotedTweet', 'mentionedUsers']
tweets = raw_tweets[cols]
tweets.rename(columns={'id':'tweetId', 'url':'tweetUrl'}, inplace=True)
#tweets.head(5)

Finally, I have created the final DataFrame for tweets that contain the hashtag "#FarmersProtest". Duplicate records are dropped from the DF based on the unique ID for each tweet (the field 'tweetId').

Let's take a look at the shape and first 5 records of the final tweets DataFrame.

In [6]:
# Convert to DataFrame, remove duplicates and keep only English tweets

tweets = pd.DataFrame(tweets)
tweets.drop_duplicates(subset=['tweetId'], inplace=True)
print("Shape: ", tweets.shape)
#tweets.head(5)

Shape:  (417511, 14)


# Conclusion

Hence, in this notebook we have seen how to perform some transformations to convert the raw JSON data about tweets scraped using snscrape into a more usable falt table form. The single JSON file containing data about tweets is now converted into 2 easier to use DataFrames, 'tweets' and 'users', which contain data about tweets and the users who posted those tweets separately. The 2 DFs can be joined on the 'userId' field.

From here, we can save the 'tweets' and 'users' DataFrames as CSV files or continue the analysis using the DFs.

In [7]:


tweets

,tweetUrl,date,renderedContent,tweetId,userId,replyCount,retweetCount,likeCount,quoteCount,source,media,retweetedTweet,quotedTweet,mentionedUsers
0,https://twitter.com/ShashiRajbhar6/status/1376...,2021-03-30 03:33:46+00:00,Support 👇\n\n#FarmersProtest,1376739399593910273,1015969769760096256,0,0,0,0,"<a href=""http://twitter.com/download/android"" ...",None,NaN,None,None
1,https://twitter.com/kaursuk06272818/status/137...,2021-03-30 03:33:23+00:00,Supporting farmers means supporting our countr...,1376739306287427584,1332937272581263362,0,0,0,0,"<a href=""http://twitter.com/download/android"" ...",[{'previewUrl': 'https://pbs.twimg.com/media/E...,NaN,None,None
2,https://twitter.com/kaursuk06272818/status/137...,2021-03-30 03:31:00+00:00,Support farmers if you are related to food #St...,1376738704128020488,1332937272581263362,0,0,0,0,"<a href=""http://twitter.com/download/android"" ...",[{'previewUrl': 'https://pbs.twimg.com/media/E...,NaN,None,None
3,https://twitter.com/SukhdevSingh_/status/13767...,2021-03-30 03:30:45+00:00,#StopHateAgainstFarmers support #FarmersProtes...,1376738640542400518,1308356658582618112,0,1,3,0,"<a href=""http://twitter.com/download/android"" ...",None,NaN,None,None
4,https://twitter.com/Davidmu66668113/status/137...,2021-03-30 03:30:30+00:00,"You hate farmers I hate you, \nif you love the...",1376738579171344386,1357311756532649985,0,0,1,0,"<a href=""http://twitter.com/download/android"" ...",None,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419502,https://twitter.com/TeamSOOUDA/status/13560306...,2021-02-01 00:04:26+00:00,We are calling on leaders to speak up against ...,1356030613912870916,1291144723566956545,0,3,8,0,"<a href=""http://twitter.com/download/iphone"" r...",[{'previewUrl': 'https://pbs.twimg.com/media/E...,NaN,None,None
419503,https://twitter.com/VibhaBurman/status/1356030...,2021-02-01 00:04:16+00:00,Power💪🏻 of panchayat but our government still ...,1356030569709113346,1394228371,0,0,0,0,"<a href=""http://twitter.com/download/android"" ...",None,NaN,{'url': 'https://twitter.com/KisanEktaMarch/st...,None
419504,https://twitter.com/NavJammu/status/1356029858...,2021-02-01 00:01:26+00:00,@AmrinderSingh__ @CNN @cnni I’m coming to term...,1356029858699046912,546937543,0,0,1,0,"<a href=""http://twitter.com/download/iphone"" r...",None,NaN,None,"[{'username': 'AmrinderSingh__', 'displayname'..."
419505,https://twitter.com/Atwal72/status/13560296726...,2021-02-01 00:00:42+00:00,What about the peaceful protests in Delhi........,1356029672614719492,751374404,0,0,0,0,"<a href=""http://twitter.com/download/android"" ...",None,NaN,{'url': 'https://twitter.com/LBCNews/status/13...,None


In [8]:
def most_retweets():
    df1 = tweets.sort_values('retweetCount', ascending=False).head(10)
    print("Se retorna todos los datos del tweet (10 primeros) en orden de mayor a menor en términos de la cantidad de retweets")
    return df1
#most_retweets()

In [9]:
def top_users():
    df2 = tweets['userId'].value_counts()
    print("Se retorna el user id y la cantidad de tweets que emitieron")
    return df2.head(10)


In [10]:
def popular_dates():
    #df3 = tweets['date'].dt.strftime('%m/%d/%Y')
    #df3
    df3 = tweets
    df3['date'] = df3['date'].dt.strftime('%m/%d/%Y')

    #df3 = tweets
    #df3['date'] = pd.to_datetime(df3['date']).dt.normalize()
    print("Se muestra el día y su respectiva cantidad de tweets, ordenado de mayor a menor")
    df3 = df3['date'].value_counts().head(10)
    return df3
#popular_dates()

In [11]:
def popular_hashtags():
    df4 = tweets
    tweets_list = df4['renderedContent'].to_list()
    hash_counts = Counter(re.findall(r'#([a-z0-9]+)', ' '.join(tweets_list), re.I))
    #print(hash_counts)
    result = dict(Counter(hash_counts).most_common(10))

    return result
#popular_hashtags()

In [17]:
def main(option):

    if option == 1:
        most = most_retweets()
        return most
    elif option == 2:
        top = top_users()
        return top
    elif option == 3:
        popular = popular_dates()
        return popular
    elif option == 4:
        hash = popular_hashtags()
        return hash
    else:
        print("Las opciones son \n 1 - most_retweets\n2 - top_users\n3 - popular_dates\n4 - popular_hashtags\n Ingresa el número")

option = int(input("Ingrese una de las siguientes opciones \n - 1\n - 2\n - 3\ - 4"))
while option not in [1, 2, 3, 4]:
    option = int(input("Ingrese una de las siguientes opciones \n - 1\n - 2\n - 3\ - 4"))
main(option)

Se retorna el user id y la cantidad de tweets que emitieron


980633168981061632     7134
69027875               2088
2930682630             1991
452391771              1841
1354532795847073796    1803
48085084               1722
872085660886282240     1491
1340186972615307264    1458
1309302936317636608    1453
921732117024919552     1446
Name: userId, dtype: int64